In [55]:
from analyse_trips import *
import geopandas as gpd
import shapely as shp
import matplotlib.pyplot as plt
import pandas as pd
import json

In [38]:
gdf = load_data("mapstr_2024_04_24.geojson")
#   https://www.youtube.com/watch?v=xu1wRfUHtKg 
plot_interactive_map(gdf)

In [39]:
gdf.head()

In [40]:
# drop column a
gdf.drop(columns="userComment", inplace=True)
gdf.drop(columns="icon", inplace=True)
gdf.head()

In [70]:
def unpack_tag(tag_series: str) -> pd.DataFrame:
    list_tags = [json.loads(tag) for tag in tag_series]
    print(list_tags)
    # ta = pd.Series([x["name"] for x in list_tags], name="Type")
    tag_names = []
    for tag in list_tags:
        tag_names.append([x["name"] for x in tag])

    colors = []
    for tag in list_tags:
        colors.append([x["color"] for x in tag])

    return pd.DataFrame({"type": pd.Series(tag_names), "color": pd.Series(colors)})
    # colors = pd.Series([x["color"] for x in list_tags], name="Colors")
    # return pd.DataFrame({"type": tag_names, "colors": colors})


# gdf["tags"].apply(unpack_tag).loc[:]
unpack_tag(gdf["tags"])